In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import motion.detection as md
import man_vs_zone.clean as clean
project_dir = "../data/"

In [2]:
games = pd.read_csv(f'{project_dir}games.csv')
play_df = pd.read_csv(f'{project_dir}plays.csv')
player_plays = pd.read_csv(f'{project_dir}player_play.csv')
players = pd.read_csv(f'{project_dir}players.csv')
tracking_df = pd.read_csv(f'{project_dir}tracking_week_1.csv')

In [3]:
gid = 2022091104
pid = 3662

import animate
animate.animate_play(games, tracking_df, play_df, players, gid, pid)

In [4]:
players = clean.get_postion_groups(players)

cleaned_tracking = clean.flip_coords(tracking_df)

games = tracking_df['gameId'].unique()
play_df = play_df[play_df['gameId'].isin(games)]

play_df = play_df[play_df['pff_manZone'] != 'other']
tracking_df = tracking_df[(tracking_df['gameId'].isin(play_df['gameId'])) & (tracking_df['playId'].isin(play_df['playId']))]

tracking_df = pd.merge(tracking_df, players[['nflId', 'position_group']], on='nflId', how='left')

cleaned_tracking = clean.zero_coords(cleaned_tracking, play_df)



Now that we have have all plays from week 1 cleaned so that all plays are zeroed, we can proceed. 

First we select only pass plays:

In [5]:
play_df['playType'] = play_df['isDropback'].map({True: 'pass', False: 'run'})
plays = play_df[play_df['playType'] == 'pass']

Next, we want to only select plays where the defense is in man coverage

In [6]:
plays = plays[plays['pff_manZone'] == 'Man']

Only include plays in this new subset

In [7]:
cleaned_tracking = cleaned_tracking[(cleaned_tracking['gameId'].isin(plays['gameId'])) & (cleaned_tracking['playId'].isin(plays['playId']))]

player_plays = player_plays[(player_plays['gameId'].isin(plays['gameId'])) & (player_plays['playId'].isin(plays['playId']))]

Next, we need to identify the defender covering the targeted receiver for every play

In [8]:
# Filter target players and keep only the necessary columns
targets = player_plays[player_plays['wasTargettedReceiver'] == 1][['gameId', 'playId', 'nflId']]
targets.rename(columns={'nflId': 'target'}, inplace=True)

# Merge the target information into the plays DataFrame
plays = plays.merge(targets, on=['gameId', 'playId'], how='left')

plays.dropna(subset=['target'], inplace=True)

To identify the covering receiver, we can use our graph function; we can make a graph of the play and identify coverage at snap (the closest DB is the covering DB in man coverage)

In [9]:
covering_players = []

for index, play in plays.iterrows():  # Correct usage of iterrows
    play_graphs = clean.graph_one_play(tracking_df, play['gameId'], play['playId'], 'post')

    # Get graph at snap
    play_graph = play_graphs[0]

    specific_node_id = play['target']
    specific_edge_type = "coverage"

    # Initialize to handle cases where no covering player is found
    other_node = None

    # Find the other node in the edge
    for neighbor in play_graph.neighbors(specific_node_id):
        if play_graph[specific_node_id][neighbor].get("edge_type") == specific_edge_type:
            other_node = neighbor
            break  # Stop after finding the first match

    covering_players.append(other_node)  # Append even if None (no covering player found)

# Add covering players to the DataFrame
plays['covering'] = covering_players


In [10]:
Gs = clean.graph_one_play(tracking_df, gid, pid, 'post')
G = Gs[0]

# print all edges and edge types in G
for u, v, data in G.edges(data=True):
    print(u, v, data)

test = tracking_df[(tracking_df['gameId'] == gid) & (tracking_df['playId'] == pid)]

test = test[test['frameType'] == "SNAP"] 

test

45102.0 52980.0 {'type': 'coverage', 'distance': np.float64(9.998004800959041)}
46099.0 47834.0 {'type': 'coverage', 'distance': np.float64(9.387321236646796)}
46099.0 53439.0 {'type': 'coverage', 'distance': np.float64(11.169422545503416)}


,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event,position_group
4740497,2022091104,3662,37266.0,Jason Kelce,106,SNAP,2022-09-11 19:47:01.7,62.0,PHI,right,44.70,28.390000,0.28,0.78,0.03,49.59,10.96,ball_snap,oline
4740628,2022091104,3662,39950.0,Lane Johnson,106,SNAP,2022-09-11 19:47:01.7,65.0,PHI,right,43.87,25.140000,0.00,0.05,0.00,84.37,237.96,ball_snap,oline
4740759,2022091104,3662,43368.0,Isaac Seumalo,106,SNAP,2022-09-11 19:47:01.7,56.0,PHI,right,44.13,27.070000,0.08,0.61,0.00,39.36,142.55,ball_snap,oline
4740890,2022091104,3662,44834.0,Charles Harris,106,SNAP,2022-09-11 19:47:01.7,53.0,DET,right,45.77,33.130000,0.20,0.37,0.02,250.69,235.46,ball_snap,dline
4741021,2022091104,3662,44888.0,Alex Anzalone,106,SNAP,2022-09-11 19:47:01.7,34.0,DET,right,45.88,27.140000,0.14,1.21,0.01,282.73,266.51,ball_snap,linebacker
4741152,2022091104,3662,45102.0,Zach Pascal,106,SNAP,2022-09-11 19:47:01.7,3.0,PHI,right,43.83,22.000000,0.00,0.01,0.01,67.74,313.84,ball_snap,receiver
4741283,2022091104,3662,46099.0,Mike Hughes,106,SNAP,2022-09-11 19:47:01.7,23.0,DET,right,53.39,16.970000,0.40,0.36,0.04,282.21,108.84,ball_snap,cornerback
4741414,2022091104,3662,46118.0,Dallas Goedert,106,SNAP,2022-09-11 19:47:01.7,88.0,PHI,right,44.14,44.890000,0.00,0.00,0.01,98.00,151.98,ball_snap,tight_end
4741545,2022091104,3662,46259.0,DeShon Elliott,106,SNAP,2022-09-11 19:47:01.7,5.0,DET,right,45.88,34.590000,0.07,0.02,0.02,250.32,309.67,ball_snap,safety
4741676,2022091104,3662,46302.0,Jordan Mailata,106,SNAP,2022-09-11 19:47:01.7,68.0,PHI,right,43.65,30.950000,0.02,0.20,0.01,90.68,277.62,ball_snap,oline
